In [39]:
#Global Settings and Imports
import os
import sys
import numpy as np
#import lightgbm as lgb
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss

#from chatbot import chatbot_data
import lightgbm_utils as lgb_utils
#from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
#print("LightGBM version: {}".format(lgb.__version__))

System version: 3.11.5 (v3.11.5:cce6ba91b3, Aug 24 2023, 10:50:31) [Clang 13.0.0 (clang-1300.0.29.30)]


In [7]:
#Parameters Setting
MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [8]:
params = {
    "task": "train",
    "boosting_type": "gbdt",
    "num_class": 1,
    "objective": "binary",
    "metric": METRIC,
    "num_leaves": MAX_LEAF,
    "min_data": MIN_DATA,
    "boost_from_average": True,
    # set it according to your cpu cores.
    "num_threads": 20,
    "feature_fraction": 0.8,
    "learning_rate": TREE_LEARNING_RATE,
}

In [24]:
# Specify the path to your CSV file
csv_file_path = "/Users/marianareyes/Documents/GitHub/chatbots/chatbot/data.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df.head()

,name,gender,age,year,major,nationality,languages,hobbies
0,John Smith,Male,20,2,Computer Science and Artificial Intelligence,USA,English,"Playing video games, reading, hiking"
1,Emily Johnson,Female,21,3,Business Administration,Canada,"English, French","Painting, playing guitar, photography"
2,Michael Williams,Male,22,4,Economics,UK,English,"Playing football, watching movies, traveling"
3,Sarah Brown,Female,20,1,Communication and Digital Media,Australia,English,"Writing, photography, dancing"
4,David Jones,Male,19,1,Architecture,Germany,"German, English","Drawing, playing piano, cooking"


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         314 non-null    object
 1   gender       314 non-null    object
 2   age          314 non-null    int64 
 3   year         314 non-null    int64 
 4   major        314 non-null    object
 5   nationality  314 non-null    object
 6   languages    314 non-null    object
 7   hobbies      314 non-null    object
dtypes: int64(2), object(6)
memory usage: 19.8+ KB


In [26]:
#Encoding categorical variables

#One-hot encoding gender
gender_encoder = OneHotEncoder()
gender_encoded = gender_encoder.fit_transform(df[['gender']])
gender_encoded_df = pd.DataFrame(gender_encoded.toarray(), columns=gender_encoder.categories_[0])


#Label encoding major, nationality, language, and hobbies
label_encoder = LabelEncoder()
df['major_encoded'] = label_encoder.fit_transform(df['major'])
df['nationality_encoded'] = label_encoder.fit_transform(df['nationality'])
df['language_encoded'] = label_encoder.fit_transform(df['languages'])
df['hobbies_encoded'] = label_encoder.fit_transform(df['hobbies'])

# Concatenate the one-hot encoded gender DataFrame with the original DataFrame
df = pd.concat([df, gender_encoded_df], axis=1)

# Drop the original categorical columns
df.drop(['gender', 'major', 'nationality', 'languages', 'hobbies'], axis=1, inplace=True)

df.head()

,name,age,year,major_encoded,nationality_encoded,language_encoded,hobbies_encoded,Female,Male
0,John Smith,20,2,5,33,4,39,0.0,1.0
1,Emily Johnson,21,3,3,5,7,14,1.0,0.0
2,Michael Williams,22,4,8,32,4,23,0.0,1.0
3,Sarah Brown,20,1,4,1,4,52,1.0,0.0
4,David Jones,19,1,1,10,11,7,0.0,1.0


In [31]:
#Splitting the data into 3 sets
# split data to 3 sets    
length = len(df)
train_data = df.loc[:0.8*length-1]
valid_data = df.loc[0.8*length:0.9*length-1]
test_data = df.loc[0.9*length:]

In [32]:
# Splitting features and target variables
train_x, train_y = train_data.drop(columns=['name']), train_data['name']
valid_x, valid_y = valid_data.drop(columns=['name']), valid_data['name']
test_x, test_y = test_data.drop(columns=['name']), test_data['name']

In [34]:
# Categorical feature columns
cate_cols = ['major_encoded', 'nationality_encoded', 'language_encoded', 'hobbies_encoded']

# Creating LightGBM datasets
lgb_train = lgb.Dataset(train_x, train_y, params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y, reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y, reference=lgb_train, categorical_feature=cate_cols)

# Training the LightGBM model
NUM_OF_TREES = 1000  # Number of boosting rounds
EARLY_STOPPING_ROUNDS = 10  # Early stopping rounds
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      valid_sets=[lgb_train, lgb_valid],
                      categorical_feature=cate_cols,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS)

# Evaluation on test data
test_predictions = lgb_model.predict(test_x)

NameError: name 'lgb' is not defined